In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from metric import *
import os 

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]="7"  # Set the GPUs 2 and 3 to use
SEED = 10
M = 5

### Data Generation and fitting

In [4]:
for N in [1000, 10000, 2000, 20000]:
    X_obs, X_mis, mask, y = make_data(n=N, seed=SEED)
    np.save(f'exp{N}', {'X':X_obs, 'y':y})
    print(y.mean())

0.566
0.5515
0.5525
0.56105


In [3]:
# N = 2000
# mcar = np.load(f'/home/redgreen/ssl/metric/MCAR_exp{N}.npy', allow_pickle=True).item()
# mar = np.load(f'/home/redgreen/ssl/metric/MAR_exp{N}.npy', allow_pickle=True).item()
# mnar = np.load(f'/home/redgreen/ssl/metric/MNAR_exp{N}.npy', allow_pickle=True).item()
# print(mnar['test']['mask'][0] == mnar['test']['mask'][0])

N = 'adult'
mcar = np.load(f'/home/redgreen/ssl/metric/MCAR_{N}.npy', allow_pickle=True).item()
mar = np.load(f'/home/redgreen/ssl/metric/MAR_{N}.npy', allow_pickle=True).item()
mnar = np.load(f'/home/redgreen/ssl/metric/MNAR_{N}.npy', allow_pickle=True).item()

In [4]:
adult = np.load('/home/redgreen/ssl/metric/adult.npy', allow_pickle=True).item()
GAIN = np.load('/home/redgreen/ssl/metric/GAIN_adult.npy', allow_pickle=True).item()
col_names = adult['col_names']

In [23]:
print(col_names)

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')


In [5]:
print(sum(mnar['test']['mask'] != mcar['test']['mask']))
print(sum(mar['test']['mask'] != mcar['test']['mask']))
print(sum(mnar['test']['mask'] != mar['test']['mask']))

#결측치 비율
pd.DataFrame([1-mcar['test']['mask'].mean(axis=0), 
              1-mar['test']['mask'].mean(axis=0), 
              1-mnar['test']['mask'].mean(axis=0),],
             index=['MCAR', 'MAR', 'MNAR'],
            #  columns=['x1', 'x2', 'x3']
            columns=col_names
             ).round(3)

[4653 4650 4667 4727 4675 4688 4667 4704 4637 4730 4755 4699 4618 4684]
[4653 2948 4667 4727 4675 2832 4667 4704 4637 4730 4755 2965 2914 4684]
[   0 2914    0    0    0 2896    0    0    0    0    0 2912 2888    0]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
MCAR,0.199,0.201,0.195,0.203,0.202,0.193,0.194,0.203,0.201,0.204,0.202,0.202,0.199,0.202
MAR,0.198,0.000,0.198,0.206,0.195,0.000,0.200,0.198,0.195,0.200,0.204,0.000,0.000,0.203
MNAR,0.198,0.199,0.198,0.206,0.195,0.198,0.200,0.198,0.195,0.200,0.204,0.199,0.197,0.203


In [ ]:
'''
[process]
1. upload.py의 upload_data/loader 함수에 데이터를 추가한다.
2. defaults.yaml, dataloader.yaml 파일을 만든다 (mech별)
3. 모델을 학습한다 (mech별) -> outputs폴더의 결과 폴더 이름을 바꾼다
4. for_prediction.ipynb에서  (mech별) 사용된 데이터셋과 예측값을 저장한다.
5. metric.ipynb에서 4의 데이터셋을 불러와 MICE를 학슶시키고 두 모델의 예측값을 비교한다.
'''

In [6]:
# adult
cat_ids = [1, 3, 5, 6, 7, 8, 9, 13]
num_ids = [0, 2, 4, 10, 11, 12]

# # credit
# cat_cols = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# num_cols = [0, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [141]:
# from sklearn.impute import KNNImputer
# model = KNNImputer(n_neighbors=10)

# model.fit(X)

# np.mean(np.maximum(model.transform(X)[:, cat_ids].round()[mask[:, cat_ids]], 0) == X[:, cat_ids][mask[:, cat_ids]])

0.891058523272856

In [9]:
# from sklearn.model_selection import train_test_split
# ind = np.arange(N)
# ind_train, ind_test = train_test_split(ind, test_size=0.3)
mode = 'test'
for data, miss_algo in zip([mcar, mar, mnar], ['MCAR', 'MAR', "MNAR"]):
    print(f'>>>>> {miss_algo} >>>>>>')
    X = data[mode]['X']
    y = data[mode]['y']
    mask = (1-data[mode]['mask']).astype(bool)
    # X = data['test']['X']
    # y = data['test']['y']
    # mask = (1-data['test']['mask']).astype(bool)
    X_mis = X.copy()
    X_mis[mask] = np.nan

    # MICE
    imputer = Imputer(seed=SEED, M=M, num_ids=num_ids, cat_ids=cat_ids)
    imputer.fit(X_mis)
    X_mice = imputer.transform(X_mis)

    # GAIN
    X_GAIN = GAIN[miss_algo]
    ######

    # EGG-GAE
    X_egg = data[mode]['rec_x']
    
    
    # Distributions of Imputed values
    df_dist, Xs_masked_inv = make_dist_df(Xs = [X, X_mice, X_egg, X_GAIN], 
                                      names=['origin', 'MICE', 'EGG-GAE', 'GAIN'], 
                                      mask=mask, index=col_names)
    
    display(df_dist)
    
    # Scores of imputed values
    # score by each X
    all_score = {'num':{}, 'cat':{}}
    for name in ['MICE', 'EGG-GAE', 'GAIN']:
        print(f'\t[{miss_algo}/{name}]')
        for ids, feat_mode in zip([num_ids, cat_ids], 
                        ['num', 'cat']):
            print(f'\t>>>[{miss_algo}/{name}/{feat_mode}]')
            df_score = evaluate_X_each(Xs_masked_inv['origin'][:, ids],
                                    Xs_masked_inv[name][:, ids],
                                    y,
                                    mask[:, ids], feat_mode, index=col_names[ids])
            display(df_score)
            all_score[feat_mode][name] = df_score.mean()
    for feat_mode in ['num', 'cat']:
        print(f'\t>>>[{miss_algo}//{feat_mode}]')
        all_score_df = pd.DataFrame(all_score[feat_mode])
        display(all_score_df)
    # # score all X
    # for ids, feat_mode in zip([num_ids, cat_ids], 
    #                     ['num', 'cat']):
    #     print(f'\t>>>[{miss_algo}//{feat_mode}]')
    #     df_score = evaluate_X(Xs_masked_inv['origin'][:, ids],
    #                         [
    #                             Xs_masked_inv['MICE'][:, ids], 
    #                             Xs_masked_inv['EGG-GAE'][:, ids],
    #                             Xs_masked_inv['GAIN'][:, ids]
    #                         ],
    #                         y,
    #                         mask[:, ids], feat_mode, names=['MICE', 'EGG-GAE', 'GAIN'])
    #     display(df_score)

>>>>> MCAR >>>>>>
0 0
>> 1 0
0 1
>> 1 2
1 1
>> 3 2
1 2
>> 3 4
2 2
>> 5 4
2 3
>> 5 10
3 3
>> 6 10
4 3
>> 7 10
5 3
>> 8 10
6 3
>> 9 10
7 3
>> 13 10
7 4
>> 13 11
7 5
>> 13 12
7 6
>> 13 -1


,origin(mean),origin(std),MICE(mean),MICE(std),EGG-GAE(mean),EGG-GAE(std),GAIN(mean),GAIN(std)
age,0.046387,1.133619,-0.009896,0.591392,0.109777,0.697344,-0.045119,0.090875
workclass,4.091927,1.554755,4.013501,0.553594,3.962008,0.984237,4.085482,0.335819
fnlwgt,-0.002294,1.050217,0.027701,0.514807,0.053912,0.172355,-0.069582,0.318967
education,10.327952,3.848075,10.501009,1.658587,10.590985,3.573778,10.212580,0.423673
education-num,0.010114,1.103539,0.022449,0.609333,0.028642,0.982171,-0.005416,0.097208
marital-status,2.600636,1.507434,2.527119,0.646562,2.521893,1.352003,2.083333,0.276385
occupation,6.784632,4.269285,6.963976,1.559708,4.828692,3.770208,6.470215,0.541129
relationship,1.405977,1.592690,1.352653,0.907943,1.081934,1.409455,0.978173,0.227095
race,3.677310,0.833944,3.723913,0.264470,3.979620,0.246422,3.989470,0.102074
sex,0.682093,0.465663,0.684574,0.257920,0.691818,0.461740,0.743461,0.436723


	[MCAR/MICE]
	>>>[MCAR/MICE/num]


,mae,mse,r2,mean_sim,std_sim,corr
age,0.941304,1.387610,-0.079774,1.213338,0.478315,0.186747
fnlwgt,0.900814,1.365601,-0.238128,13.075678,0.509809,0.003034
education-num,0.892154,1.252707,-0.028665,-1.219604,0.447837,0.250238
capital-gain,0.587317,2.182016,-0.111169,0.501901,0.629373,0.036046
capital-loss,0.721599,1.708195,-0.198196,0.296700,0.545551,0.009681
hours-per-week,0.842182,1.350063,-0.076756,0.042965,0.470205,0.192459


	>>>[MCAR/MICE/cat]


...

,mae,mse,r2,acc,mle_rel
workclass,0.803020,2.402196,-0.004384,0.685999,0.843903
education,2.620119,14.228987,-0.035020,0.293514,0.936852
marital-status,1.348412,2.441644,-0.091517,0.035912,9.957587
occupation,3.772169,18.459072,0.019623,0.055252,1.816047
relationship,1.303658,2.354037,0.094219,0.121118,3.553982
race,0.343717,0.863143,-0.158579,0.850543,0.835144
sex,0.313351,0.313351,-0.502776,0.686649,0.191114
native-country,2.763070,57.279121,-0.118346,0.468193,13.280029


	>>>[MNAR//num]


,MICE,EGG-GAE,GAIN
mae,0.800963,0.540404,0.874845
mse,1.483565,1.066159,2.802971
r2,-0.114750,0.218417,-0.807049
mean_sim,1.255016,8.260100,4.870857
std_sim,0.504663,0.599967,0.490601
corr,0.126415,0.378265,0.102815


	>>>[MNAR//cat]


,MICE,EGG-GAE,GAIN
mae,1.635891,1.215148,1.658439
mse,11.166275,12.248214,12.292694
r2,0.123169,0.012032,-0.099597
acc,0.446027,0.715516,0.399647
mle_rel,138548.533059,11622.766715,3.926832


### Distributions of Imputed values

In [ ]:
def make_cnt_vec(key, cnt, size):
    arr = np.empty(size)
    for k, c in zip(key, cnt):
        arr[k] = c
    return arr

In [84]:
for name in ['MICE', 'EGG-GAE']:
    print(f'\t[{name}]')
    for ids, mode in zip([num_ids, cat_ids], 
                    ['num', 'cat']):
        print(f'\t>>>[{mode}]')
        df_score = evaluate_X_each(Xs_masked_inv['origin'][:, ids],
                                Xs_masked_inv[name][:, ids],
                                y,
                                mask[:, ids], mode, index=col_names[ids])
        display(df_score)

	[MICE]
	>>>[num]


,mae,mse,r2,mean_sim,std_sim,corr
age,0.947239,1.405790,-0.094368,0.807391,0.475131,0.173557
fnlwgt,0.893782,1.334722,-0.210793,9.219243,0.513300,0.027504
capital-gain,0.578751,2.156595,-0.101088,0.559502,0.628978,0.050162
capital-loss,0.714464,1.672018,-0.174595,0.196982,0.545387,0.035507
hours-per-week,0.849970,1.384189,-0.103132,0.251490,0.457722,0.176057


	>>>[cat]


,mae,mse,r2,acc,mle_rel
workclass,1.193487,2.698010,-0.116725,0.0,67.365325
education,3.217566,16.191633,-0.092296,0.0,34975.982075
education-num,0.891863,1.251750,-0.028719,0.0,3477.075211
marital-status,1.220085,2.474934,-0.089227,0.0,13.656192
occupation,3.795323,20.205805,-0.108785,0.0,95434.586796
relationship,1.138454,2.043059,0.195101,0.0,1.435822
race,0.626808,0.827172,-0.189261,0.0,302747.131750
sex,0.343101,0.185188,0.146138,0.0,1856.449784
native-country,4.582798,58.418060,-0.205371,0.0,NaN


	[EGG-GAE]
	>>>[num]


,mae,mse,r2,mean_sim,std_sim,corr
age,0.703381,0.805480,0.372957,-1.384518,0.385112,0.613274
fnlwgt,0.785977,1.067657,0.031474,19.094839,0.835532,0.186899
capital-gain,0.290266,1.859290,0.050706,1.119341,0.866377,0.266328
capital-loss,0.489904,1.390238,0.023356,0.349690,0.893753,0.166086
hours-per-week,0.697817,1.044012,0.167973,-1.379113,0.624514,0.411823


	>>>[cat]


,mae,mse,r2,acc,mle_rel
workclass,0.588779,1.487341,0.384379,0.737468,0.667181
education,1.056675,6.370317,0.570255,0.735496,0.567532
education-num,0.299447,0.258531,0.787533,0.000000,222657.683305
marital-status,0.803895,3.015545,-0.327152,0.747932,0.179228
occupation,3.616478,27.058458,-0.484821,0.349348,2.714708
relationship,0.701187,1.952511,0.230774,0.701522,0.371992
race,0.315282,0.775258,-0.114622,0.860947,0.786749
sex,0.190890,0.190890,0.119850,0.809110,0.024648
native-country,1.917457,51.110542,-0.054591,0.898042,20.911270


In [23]:
df_dist, Xs_masked_inv = make_dist_df(Xs = [X, X_mice, X_egg], 
                                      names=['origin', 'MICE', 'EGG-GAE'], mask=mask)

df_dist

/tmp/ipykernel_2257613/3804797055.py:17: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(masked_inv, axis=0)
/home/redgreen/miniconda3/envs/ssl/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


,origin(mean),origin(std),MICE(mean),MICE(std),EGG-GAE(mean),EGG-GAE(std)
x1,0.583716,1.396067,0.536104,0.876596,0.677885,0.640420
x2,NaN,NaN,NaN,NaN,NaN,NaN
x3,-0.344322,1.069565,-0.483261,0.809786,0.563412,0.815381


### Scores of imputed values

In [612]:
# score by each X
df_score = evaluate_X_each(Xs_masked_inv['origin'],
                     Xs_masked_inv['MICE'],
                     mask)

df_score

,mae,mse,mean_sim,std_sim,r2,corr,rel_sim
x1,0.648871,0.754029,-0.010979,0.291406,0.254106,0.533681,0.002562
x2,0.837852,1.112965,-0.005985,0.340622,0.108955,0.412356,0.000000
x3,1.046552,1.857424,0.001914,0.262980,0.332165,0.593870,0.004486


In [614]:
# score all X
df_score = evaluate_X(Xs_masked_inv['origin'],
                     [
                         Xs_masked_inv['MICE']
                      ],
                     mask, names=['MICE'])

df_score

,MICE
mae,0.844944
mse,1.243606
mean_sim,-0.001695
std_sim,0.121495
r2,0.667081
corr,0.818923
F1_sim,-0.002135
